# Using Imagery Layers 

The `arcgis.raster` module contains classes and raster analysis functions for working with raster data and imagery layers.

Raster data is made up of a grid of cells, where each cell or pixel can have a value. Raster data is useful for storing data that varies continuously, as in a satellite image, a surface of chemical concentrations, or an elevation surface.

The `Imagerylayer` class represents an image service resource as a layer. An `ImageryLayer` object retrieves and displays data from image services. `ImageryLayer` allows you to and apply server defined or client-defined rendering rules (e.g. remap, colormap), and mosaic rules.

## Accesssing Imagery Layers

Imagery layer items are available as content in the GIS. You can search the GIS for imagery layer items, or get them using their item id:

### Searching the GIS for imagery layer items

You can search the GIS for imagery layers by specifying the item type as `Imagery Layer`:

In [ ]:
import arcgis
from arcgis.gis import GIS
from IPython.display import display

gis = GIS()

In [ ]:
items = gis.content.search("Landsat 8 Views", item_type="Imagery Layer", max_items=2)

In [ ]:
for item in items:
    display(item)

<Item title:"Landsat 8 Views" type:Image Service owner:esri>

<Item title:"L8_Views_Preview" type:Image Service owner:esri_atlas>

### Retrieving Imagery Layer item using item id
Imagery Layers items can be obtained using their item id as well:

In [ ]:
l8_views = gis.content.get('4ca13f0e4e29403fa68c46d188c4be73')
l8_views

<Item title:"Landsat 8 Views" type:Image Service owner:esri>

### Accessing `ImageryLayer` from Imagery Layer items

Like other layer based items in the GIS, imagery layer items allow access to the imagery layer through the `layers` property of the item. Each imagery layer item has one imagery layer instance in it's `layers` property:

In [ ]:
l8_views.layers

[<ImageryLayer url:"https://landsat2.arcgis.com/arcgis/rest/services/Landsat8_Views/ImageServer">]

In [ ]:
l8_lyr = l8_views.layers[0]
l8_lyr

<ImageryLayer url:"https://landsat2.arcgis.com/arcgis/rest/services/Landsat8_Views/ImageServer">

### Creating `ImageryLayer` from an image service url

`ImageryLayer` objects can also be constructed from an image service URL:

In [ ]:
img_svc_url = 'https://landsat2.arcgis.com/arcgis/rest/services/Landsat8_Views/ImageServer'

In [ ]:
from arcgis.raster import ImageryLayer

In [ ]:
landsat_lyr = ImageryLayer(img_svc_url)

In [ ]:
landsat_lyr

<ImageryLayer url:"https://landsat2.arcgis.com/arcgis/rest/services/Landsat8_Views/ImageServer">

### Creating `ImageryLayer` from secure image services

If the imagery layer is a served by a secure image service, pass in the `GIS` object to the `ImageryLayer` constructor to specify the GIS which should be used to connect to the service.

In [ ]:
portal = GIS("portal url", "username","password", verify_cert=False)

In [ ]:
secure_url = 'https://dev003248.esri.com:6443/arcgis/rest/services/ImgSrv_Landast_Montana2015/ImageServer'

In [ ]:
secure_img_lyr = ImageryLayer(secure_url, portal)

In [ ]:
secure_img_lyr

<ImageryLayer url:"https://dev003248.esri.com:6443/arcgis/rest/services/ImgSrv_Landast_Montana2015/ImageServer">

## Properties of an `ImageryLayer` object
The `properties` property on an `ImageryLayer` object provides a dictionary representation of all its properties. However, you can access individual properties as attributes as well:

In [ ]:
landsat_lyr.properties.name

'Landsat8_Views'

In [ ]:
landsat_lyr.properties['description']

'Multispectral Landsat 8 OLI Image Service covering the landmass of the World. This service includes 8-band multispectral scenes, at 30 meter resolution. It can be used for mapping and change detection of agriculture, soils, vegetation health, water-land features and boundary studies. Using on-the-fly processing, the raw DN values are transformed to scaled (5000 - 55000) apparent reflectance values and then different service based renderings for band combinations and indices are applied. The service is updated on a daily basis to include the latest best scenes from the USGS.'

The `capabilities` property is useful to know what kinds of operations can be performed on the imagery layer.

In [ ]:
landsat_lyr.properties.capabilities

'Catalog,Mensuration,Image,Metadata'

You can access the allowed mosaic methods using the `allowedMosaicMethods` property.

In [ ]:
landsat_lyr.properties.allowedMosaicMethods

'ByAttribute,Center,NorthWest,Nadir,LockRaster,None'

## Raster Functions

Imagery Layers can be published with raster functions that can be used to change the visualization of the imagery layer using  on-the-fly image processing at display resolution. Raster functions are lightweight and process only the pixels visible on your screen, in memory, without creating intermediate files. They are powerful because you can chain them together and apply them on huge rasters and mosaics on the fly.

The code below cycles through the raster functions available with the landsat layer:

In [ ]:
for fn in landsat_lyr.properties.rasterFunctionInfos:
    print(fn['name'])

Agriculture with DRA
Bathymetric with DRA
Color Infrared with DRA
Natural Color with DRA
Short-wave Infrared with DRA
Geology with DRA
Agriculture
Bathymetric
Color Infrared
Geology
Natural Color
Short-wave Infrared
NDVI Colorized
Normalized Difference Moisture Index Colorized
NDVI Raw
NBR Raw
None


## Visualizing imagery layers

`ImageryLayer`s can be added to the map widget for visualization:

In [ ]:
map = gis.map("Pallikaranai", zoomlevel=13)
map

![raster functions in a loop](http://esri.github.io/arcgis-python-api/notebooks/nbimages/rasteranalytics.gif)

In [ ]:
map.add_layer(landsat_lyr)

### Performing on-the-fly image processing using raster functions
The utility of raster functions is better seen when we interactively cycle through these raster functions and apply them to the map, like the code below does. This is using on-the-fly image processing at display resolution to cycle through the various raster functions, and showing how the layer can be visualized using these different raster functions published with the layer:

In [ ]:
import time

for fn in landsat_lyr.properties.rasterFunctionInfos:
    print(fn['name'])
    map.remove_layers()
    map.add_layer(landsat_lyr, {"imageServiceParameters" :{ "renderingRule": { "rasterFunction": fn['name']}}})
    time.sleep(2)

Agriculture with DRA
Bathymetric with DRA
Color Infrared with DRA
Natural Color with DRA
Short-wave Infrared with DRA
Geology with DRA
Agriculture
Bathymetric
Color Infrared
Geology
Natural Color
Short-wave Infrared
NDVI Colorized
Normalized Difference Moisture Index Colorized
NDVI Raw
NBR Raw
None


### Composing raster functions
Developers can create their own raster functions, by chaining different raster functions. For instance, the code below is doing an Extract Band and extracting out the [4,5,3] band combination, and applying a Stretch to get the land-water boundary visualization that makes it easy to see where land is and where water is. Its worth noting that the raster function is applied at display resolution and only for the visible extent using on the fly image processing.

In [ ]:
# A raster function is created as a python dictionary as shown below:
def extract_bands(bands):
    return {
               "rasterFunction": "Stretch",
               "rasterFunctionArguments":{
                    "Raster":{
                        "rasterFunction": "ExtractBand",
                        "rasterFunctionArguments":{"BandIds": bands}
                    },
                    "StretchType": 6,
                    "DRA": True,
                    "Gamma": [1,1,1],
                    "UseGamma": True
                },
               "outputPixelType":"U8"
            }

Let us apply this raster function to the image layer to visualize the results. The raster function extracts the [4, 5, 3] band combination from landsat data that makes it easy to visualize where land is and where water is, as seen below:

In [ ]:
map2 = gis.map("Pallikaranai", zoomlevel=13)
map2

![landsat land water separation](http://esri.github.io/arcgis-python-api/notebooks/nbimages/04_RasterAnalytics_02.PNG)

In [ ]:
map2.add_layer(landsat_lyr, {"imageServiceParameters" : { "renderingRule": extract_bands([4, 5, 3]) }})